 sequence to sequence LSTM and GRU models for Language translation

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "hin.txt"


In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):Aakash Gangurde

    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 2979
Number of unique input tokens: 70
Number of unique output tokens: 90
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [5]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [6]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/100
38/38 [==============================] - 14s 94ms/step - loss: 1.2475 - accuracy: 0.7866 - val_loss: 1.5151 - val_accuracy: 0.6875
Epoch 2/100
38/38 [==============================] - 2s 49ms/step - loss: 0.9164 - accuracy: 0.8083 - val_loss: 1.7071 - val_accuracy: 0.6873
Epoch 3/100
38/38 [==============================] - 2s 49ms/step - loss: 0.8279 - accuracy: 0.8095 - val_loss: 1.3365 - val_accuracy: 0.6884
Epoch 4/100
38/38 [==============================] - 2s 45ms/step - loss: 0.7837 - accuracy: 0.8112 - val_loss: 1.2806 - val_accuracy: 0.6885
Epoch 5/100
38/38 [==============================] - 2s 45ms/step - loss: 0.7739 - accuracy: 0.8154 - val_loss: 1.2314 - val_accuracy: 0.6918
Epoch 6/100
38/38 [==============================] - 2s 48ms/step - loss: 0.6962 - accuracy: 0.8266 - val_loss: 1.2103 - val_accuracy: 0.7064
Epoch 7/100
38/38 [==============================] - 2s 47ms/step - loss: 0.6386 - accuracy: 0.8394 - val_loss: 1.0807 - val_accuracy: 0.7234
Epoch

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


LSTM

In [7]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



In [8]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: Wow!
Decoded sentence: क्या आपको अपना नाप पता है?

-
Input sentence: Duck!
Decoded sentence: वह काम शुर माल करता है।

-
Input sentence: Duck!
Decoded sentence: वह काम शुर माल करता है।

-
Input sentence: Help!
Decoded sentence: वह काम शुर माल करता है।

-
Input sentence: Jump.
Decoded sentence: वह काम शुर माल करता है।

-
Input sentence: Jump.
Decoded sentence: वह काम शुर माल करता है।

-
Input sentence: Jump.
Decoded sentence: वह काम शुर माल करता है।

-
Input sentence: Hello!
Decoded sentence: वह काम शुर माल करता है।

-
Input sentence: Hello!
Decoded sentence: वह काम शुर माल करता है।

-
Input sentence: Cheers!
Decoded sentence: तुम किस बाल कर सके नहर क्या हि?

-
Input sentence: Cheers!
Decoded sentence: तुम किस बाल कर सके नहर क्या हि?

-
Input sentence: Exhale.
Decoded sentence: मुझे उसकी पता जानता है।

-
Input sentence: Exhale.
Decoded sentence: मुझे उसकी पता जानता है।

-
Input sentence: Got it?
Decoded sentence: वह कमीशा दे हिए।

-
Input sentence: I'm OK.
Decoded sente

GRU

In [9]:


# Define hyperparameters
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
data_path = "hin.txt"  # Path to the data txt file on disk.

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()Aakash Gangurde


with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.GRU(latent_dim, return_state=True)
encoder_outputs, state_h = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the state.
encoder_states = [state_h]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

# Training the modelAakash Gangurde

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

# Save the model
model.save("s2s")

# Define sampling models for inference
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc = model.layers[2].output
encoder_states = [state_h_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h]
decoder_gru = model.layers[3]
decoder_outputs, state_h_dec = decoder_gru(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())




Number of samples: 2979
Number of unique input tokens: 70
Number of unique output tokens: 90
Max sequence length for inputs: 107
Max sequence length for outputs: 123
Epoch 1/100
38/38 [==============================] - 8s 73ms/step - loss: 1.3428 - accuracy: 0.7861 - val_loss: 1.8868 - val_accuracy: 0.6876
Epoch 2/100
38/38 [==============================] - 2s 40ms/step - loss: 0.9046 - accuracy: 0.8084 - val_loss: 1.6402 - val_accuracy: 0.6873
Epoch 3/100
38/38 [==============================] - 1s 39ms/step - loss: 0.7907 - accuracy: 0.8099 - val_loss: 1.2800 - val_accuracy: 0.6901
Epoch 4/100
38/38 [==============================] - 1s 38ms/step - loss: 0.7349 - accuracy: 0.8177 - val_loss: 1.1734 - val_accuracy: 0.7006
Epoch 5/100
38/38 [==============================] - 1s 39ms/step - loss: 0.6572 - accuracy: 0.8343 - val_loss: 1.0859 - val_accuracy: 0.7367
Epoch 6/100
38/38 [==============================] - 2s 42ms/step - loss: 0.6126 - accuracy: 0.8469 - val_loss: 1.0170 - val

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


In [10]:
import numpy as np

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0
    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        output_tokens, h = decoder_model.predict([target_seq] + [states_value])  # Ensure states_value is in a list
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h]  # Put h in a list

    return decoded_sentence

# Testing the model on some input sequences
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence: Wow!
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Duck!
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Duck!
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Help!
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Jump.
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Jump.
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Jump.
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Hello!
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Hello!
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Cheers!
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Cheers!
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Exhale.
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Exhale.
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: Got it?
Decoded sentence: मैं उसका दिलना चाहिए थी।

-
Input sentence: I'm OK.
Decoded sen